## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [2]:
df = pd.read_csv('citadel_acquisition_and_leases_map - Sheet1.csv')

## Image URL collector

In [52]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [4]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

In [5]:
len(links)

6

## Article link formatter

In [7]:
df["short_description"] = df["purchased_or_leased"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['purchased_or_leased'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/04...
1    <a href="https://therealdeal.com/miami/2022/06...
2    <a href="https://therealdeal.com/miami/2022/08...
3    <a href="https://therealdeal.com/miami/2022/08...
4    <a href="https://therealdeal.com/miami/2022/08...
5    <a href="https://therealdeal.com/miami/2022/07...
Name: description_link, dtype: object

In [8]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/04/21/waterfront-brickell-development-site-of-planned-supertalls-sells-for-record-363m/" target="_blank" rel="noopener noreferrer">purchased for $363</a> million'

## Google Maps API Geolocater Setup

In [9]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [11]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [59]:
df.columns

Index(['address', 'stories', 'units', 'developer', 'status', 'project_cost',
       'land_cost', 'loan_amount', 'lender', 'description', 'story link',
       'useful_info', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [12]:
df

,full_address,purchased_or_leased,description,story_link,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,"1201 Brickell Bay Drive, Birckell Miami, FL",purchased for $363 million,This is a 2.5-acre development site along the bay,https://therealdeal.com/miami/2022/04/21/water...,https://s11.therealdeal.com/trd/m/up/2022/04/W...,purchased for $363,million,"<a href=""https://therealdeal.com/miami/2022/04...","1201 Brickell Bay Drive, Birckell Miami, FL","(1201 Brickell Bay Dr, Miami, FL 33131, USA, (...","(25.7618982, -80.19118979999999, 0.0)",25.761898,-80.191190,0.0
1,"1221 Brickell Avenue, Brickell Miami, FL",purchased for $286.5 million,"If we include this, we would have to couch it ...",https://therealdeal.com/miami/2022/06/24/rockp...,https://s14.therealdeal.com/trd/m/up/2022/06/m...,purchased for $286.5,million,"<a href=""https://therealdeal.com/miami/2022/06...","1221 Brickell Avenue, Brickell Miami, FL","(1221 Brickell Ave, Miami, FL 33131, USA, (25....","(25.7612274, -80.1910809, 0.0)",25.761227,-80.191081,0.0
2,"1250 and 1260 Brickell Bay Drive, Miami, FL",purchased for $20 million,This is a small apartment building at 1250 Bri...,https://therealdeal.com/miami/2022/08/05/ken-g...,https://s12.therealdeal.com/trd/m/up/2022/08/m...,purchased for $20,million,"<a href=""https://therealdeal.com/miami/2022/08...","1250 and 1260 Brickell Bay Drive, Miami, FL","(1260 Brickell Bay Dr, Miami, FL 33131, USA, (...","(25.761018, -80.190164, 0.0)",25.761018,-80.190164,0.0
3,"830 Brickell Plaza, Miami FL","leased roughly 95,000 square feet",NaN,https://therealdeal.com/miami/2022/08/03/ken-g...,https://s11.therealdeal.com/trd/m/up/2022/08/K...,leased roughly,"95,000 square feet","<a href=""https://therealdeal.com/miami/2022/08...","830 Brickell Plaza, Miami FL","(830 Brickell Plaza, Miami, FL 33131, USA, (25...","(25.765873, -80.192504, 0.0)",25.765873,-80.192504,0.0
4,"200 South Biscayne Boulevard, Miami FL",leased,Citadel had an office there but it expanded it...,https://therealdeal.com/miami/2022/08/03/ken-g...,https://s11.therealdeal.com/trd/m/up/2022/08/K...,leased,,"<a href=""https://therealdeal.com/miami/2022/08...","200 South Biscayne Boulevard, Miami FL","(200 S Biscayne Blvd, Miami, FL 33131, USA, (2...","(25.7722422, -80.18773809999999, 0.0)",25.772242,-80.187738,0.0
5,"151 Worth Avenue in Palm Beach, FL",leased,Citadel leased the former Neiman Marcus buildi...,https://therealdeal.com/miami/2022/07/14/ken-g...,https://therealdeal.com/miami/wp-content/uploa...,leased,,"<a href=""https://therealdeal.com/miami/2022/07...","151 Worth Avenue in Palm Beach, FL","(151 Worth Ave, Palm Beach, FL 33480, USA, (26...","(26.7010839, -80.0350665, 0.0)",26.701084,-80.035066,0.0


## Correction section

In [61]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [13]:
df.columns

Index(['full_address', 'purchased_or_leased', 'description', 'story_link',
       'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [14]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
#     developer = df['developer'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [64]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [15]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/04/21/waterfront-brickell-development-site-of-planned-supertalls-sells-for-record-363m/" target="_blank" rel="noopener noreferrer">purchased for $363</a> million'

In [16]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Bay Harbor Islands Resi Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [71]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [68]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [69]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Bay_Harbor_Resi_pipeline
